In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from imblearn.over_sampling import RandomOverSampler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV

pd.set_option("display.max_columns", None) # Show all columns
pd.set_option("display.max_rows", None) # Show all rows

# Display row into about 120 text characters before wrapping to a new line
pd.set_option("display.width", 120)

# Read the CSV file
# the file is inside data/raw/ folder
df = pd.read_csv("data/raw/diabetes_prediction_dataset.csv")

# Peek at the first 5 rows
df.head() 

# ======================================= STEP 1: DATA UNDERSTANDING =======================================

# Display total rows and column
print("Shape (rows, columns):", df.shape)

# Display all column names
print("\nColumns in the dataset:")
print(df.columns.tolist())

# Check the data types of each column
print("Data types of columns:\n")
print(df.dtypes)

# Categorical columns:
# - gender
# - smoking_history

# Categorical columns (binary values)
# meaning the column has only two unique values. 
# They represent "no/yes" categories
# - heart_disease
# - hypertension  
# - diabetes (TARGET)  
# Note: They store integers (0, 1), not strings. 
# But they are still categorical, because there's only two possible values
# These two columns are categorical in meaning, but numerical in storage.

# Numerical columns (with decimal point):
# - age          
# - bmi                    
# - HbA1c_level            

# Numerical columns (integer):                       
# - blood_glucose_level      

# Check if missing value exist in every columns
print("Missing values per column:\n")
print(df.isna().sum())

# Check if duplicates exist or not
print("Number of duplicate rows:", df.duplicated().sum())

# Count how many people have diabetes (1) and not (0)
print("Diabetes value counts:\n")
print(df["diabetes"].value_counts())

# Display as percentage too
print("\nPercentage distribution:")
print((df["diabetes"].value_counts(normalize=True) * 100).round(2))

# Count how many have diabetes (1) vs not (0)
counts = df["diabetes"].value_counts()

# Create the pie chart
plt.figure(figsize=(5,5))
plt.pie(counts, labels=["No Diabetes (0)", "Diabetes (1)"], autopct="%.1f%%", startangle=90, colors=["#6AB7FF", "#FF9999"])
plt.title("Diabetes Distribution")
plt.show()

# ======================================= STEP 2: EDA (Exploratory Data Analysis) =======================================

# Check unique values in the two categorical columns
print("Unique values in 'gender':")
print(df["gender"].unique())

print("\nUnique values in 'smoking_history':")
print(df["smoking_history"].unique())

# Count how many samples belong to each category

print("Gender value counts:")
print(df["gender"].value_counts())

print("\nSmoking history value counts:")
print(df["smoking_history"].value_counts())

print("\n")

# Make the figure
plt.figure(figsize=(12,5))

# --- Gender count plot ---
plt.subplot(1,2,1)  # first plot (1 row, 2 columns, position 1)
df["gender"].value_counts().plot(kind="bar", color=["#C93C20", "#924E7D", "#00821a"])
plt.title("Gender Distribution")
plt.xlabel("Gender")
plt.ylabel("Count")

# --- Smoking history plot ---
plt.subplot(1,2,2)  # second plot (position 2)
df["smoking_history"].value_counts().plot(kind="bar", color=["#8E402A", "#317F43", "#1F3A3D", "#3B83BD", "#686C5E", "#7FB5B5"])
plt.title("Smoking History Distribution")
plt.xlabel("Smoking History")
plt.ylabel("Count")

plt.tight_layout()
plt.show()

# Display total of 0s and 1s 
# for hypertension and heart_disease
# Reminder, these two columns (hypertension and heart_disease)
# are categorical columns. They store numeric values, but are categorical in meaning.

print("Hypertension value counts:")
print(df["hypertension"].value_counts().sort_index())
print()

print("Heart disease value counts:")
print(df["heart_disease"].value_counts().sort_index())

print("\n")

binary_cols = ["hypertension", "heart_disease"]
plt.figure(figsize=(10,4))
for i, col in enumerate(binary_cols, 1):
    plt.subplot(1, 2, i)
    df[col].value_counts().sort_index().plot(kind="bar", color=["#6AB7FF", "#FF9999"], edgecolor="black")
    plt.title(f"{col} value counts")
    plt.xlabel(col)
    plt.ylabel("Count")
    plt.xticks([0, 1], ["0 (No)", "1 (Yes)"])
plt.tight_layout()
plt.show()

# See how continuous columns like 
# age, bmi, HbA1c_level, blood_glucose_level) are spread out.
# It helps detect outliers and skewed data
numeric_cols = ["age", "bmi", "HbA1c_level", "blood_glucose_level"]

df[numeric_cols].hist(figsize=(10,6), bins=30, color="#90CAF9", edgecolor="black")
plt.suptitle("Distribution of Numeric Features", y=1.02)
plt.tight_layout()
plt.show()

# ======================================= STEP 3: DATA PREPROCESSING =======================================

# Remove duplicates

# Count total rows in the dataset
before = len(df)

print(f"Current rows: {before}")

# Display total duplicates
print("Number of duplicate rows: ", df.duplicated().sum())

# Remove the duplicates
df = df.drop_duplicates()

# Count total rows in the dataset after removing duplicate rows
after = len(df)

print(f"Removed {before - after} duplicate rows.")
print(f"Remaining rows: {after}")

# Check for invalid values (0 or negative) in key numeric columns

invalid_rows = df[
    (df['age'] <= 0) |
    (df['bmi'] <= 0) |
    (df['HbA1c_level'] <= 0) |
    (df['blood_glucose_level'] <= 0)
]

print("Number of rows with invalid values:", len(invalid_rows))
invalid_rows.head()

# Detect outliers using IQR method for key numeric columns

cols_to_check = ["age", "bmi", "HbA1c_level", "blood_glucose_level"]

for col in cols_to_check:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    
    outliers = df[(df[col] < lower) | (df[col] > upper)]
    
    print(f"{col}:")
    print(f"  Q1 = {Q1:.2f}, Q3 = {Q3:.2f}, IQR = {IQR:.2f}")
    print(f"  Lower bound = {lower:.2f}, Upper bound = {upper:.2f}")
    print(f"  Outlier count = {len(outliers)}")
    print("-" * 50)

# Check unique values and their counts for categorical columns

cat_cols = ["gender", "smoking_history"]

for col in cat_cols:
    print(f"\nColumn: {col}")
    print(df[col].value_counts())

# Remove rows where gender is 'Other'
# because this value is the lowest compared to Male and Female

before = len(df)
df = df[df["gender"] != "Other"]
after = len(df)

print(f"Removed {before - after} rows with gender='Other'.")
print(f"Remaining rows: {after}")
print("\nUpdated gender counts:")
print(df["gender"].value_counts())

# Save the preprocessed data as a new csv file
processed_dir = Path("data/processed")

out_path = processed_dir / "preprocessed_diabetes_prediction_dataset.csv"
df.to_csv(out_path, index=False)

print("Saved to:", out_path.resolve())

df_preprocessed = pd.read_csv("data/processed/preprocessed_diabetes_prediction_dataset.csv")

print("Shape (rows, columns):", df_preprocessed.shape)
df_preprocessed.head()

del df

# Count how many people have diabetes (1) and not (0)
print("Diabetes value counts:\n")
print(df_preprocessed["diabetes"].value_counts())

# Display as percentage too
print("\nPercentage distribution:")
print((df_preprocessed["diabetes"].value_counts(normalize=True) * 100).round(2))

# Count how many have diabetes (1) vs not (0)
counts = df_preprocessed["diabetes"].value_counts()

# Create the pie chart
plt.figure(figsize=(5,5))
plt.pie(counts, labels=["No Diabetes (0)", "Diabetes (1)"], autopct="%.1f%%", startangle=90, colors=["#6AB7FF", "#FF9999"])
plt.title("Diabetes Distribution")
plt.show()

# Count how many samples belong to each category

print("Gender value counts:")
print(df_preprocessed["gender"].value_counts())

print("\nSmoking history value counts:")
print(df_preprocessed["smoking_history"].value_counts())

print("\n")

# Make the figure
plt.figure(figsize=(12,5))

# --- Gender count plot ---
plt.subplot(1,2,1)  # first plot (1 row, 2 columns, position 1)
df_preprocessed["gender"].value_counts().plot(kind="bar", color=["#C93C20", "#924E7D", "#00821a"])
plt.title("Gender Distribution")
plt.xlabel("Gender")
plt.ylabel("Count")

# --- Smoking history plot ---
plt.subplot(1,2,2)  # second plot (position 2)
df_preprocessed["smoking_history"].value_counts().plot(kind="bar", color=["#8E402A", "#317F43", "#1F3A3D", "#3B83BD", "#686C5E", "#7FB5B5"])
plt.title("Smoking History Distribution")
plt.xlabel("Smoking History")
plt.ylabel("Count")

plt.tight_layout()
plt.show()

# Display total of 0s and 1s 
# for hypertension and heart_disease
# Reminder, these two columns (hypertension and heart_disease)
# are categorical columns. They store numeric values, but are categorical in meaning.

print("Hypertension value counts:")
print(df_preprocessed["hypertension"].value_counts().sort_index())
print()

print("Heart disease value counts:")
print(df_preprocessed["heart_disease"].value_counts().sort_index())

print("\n")

binary_cols = ["hypertension", "heart_disease"]
plt.figure(figsize=(10,4))
for i, col in enumerate(binary_cols, 1):
    plt.subplot(1, 2, i)
    df_preprocessed[col].value_counts().sort_index().plot(kind="bar", color=["#6AB7FF", "#FF9999"], edgecolor="black")
    plt.title(f"{col} value counts")
    plt.xlabel(col)
    plt.ylabel("Count")
    plt.xticks([0, 1], ["0 (No)", "1 (Yes)"])
plt.tight_layout()
plt.show()

# See how continuous columns like 
# age, bmi, HbA1c_level, blood_glucose_level) are spread out.
# It helps detect outliers and skewed data
numeric_cols = ["age", "bmi", "HbA1c_level", "blood_glucose_level"]

df_preprocessed[numeric_cols].hist(figsize=(10,6), bins=30, color="#90CAF9", edgecolor="black")
plt.suptitle("Distribution of Numeric Features", y=1.02)
plt.tight_layout()
plt.show()

# ======================================= STEP 4: DATA TRANSFORMATION =======================================

# Encode gender column
# This column has two possible values only (Male and Female. Other value has been removed)
# Therefore, we can use binary encoding (0 or 1)

# Encode gender.
# Female = 0, Male = 1
df_preprocessed["gender"] = df_preprocessed["gender"].map({"Female": 0, "Male": 1})

print(df_preprocessed["gender"].value_counts())

df_preprocessed.head()

# One-hot encode smoking_history column
# Meaning it turns text into several 0/1 columns
df_preprocessed = pd.get_dummies(
    df_preprocessed,
    columns=["smoking_history"],
    drop_first=True  # drop one category to avoid redundancy
)

# Display the new columns created
print([c for c in df_preprocessed.columns if c.startswith("smoking_history_")])

df_preprocessed.head()

# Check correlation between all numeric features and the target (diabetes)
# .corr() only works with numbers, so this is done after encoding categorical columns
# (like gender and smoking_history) into numeric form
# Also, we are doing correlation for all columns in the dataset, because they are all numerical columns now.

corr = df_preprocessed.corr(numeric_only=True)["diabetes"].sort_values(ascending=False)
print(corr)

# Besides using correlation, 
# use boxplots to see if the numerical columns affect the target variable or not

# Boxplots: do these numeric features differ between diabetes=0 and 1?
numeric_cols = ["age", "bmi", "HbA1c_level", "blood_glucose_level"]

plt.figure(figsize=(12, 8))
for i, col in enumerate(numeric_cols, 1):
    plt.subplot(2, 2, i)
    sns.boxplot(data=df_preprocessed, x="diabetes", y=col, color="#90CAF9") 
    plt.xlabel("Diabetes (0 = No, 1 = Yes)")
    plt.ylabel(col)
    plt.title(f"{col} vs Diabetes")
plt.tight_layout()
plt.show()

# ======================================= STEP 5: TRAINING & TESTING =======================================

# Save the transformed dataset as a new csv file
processed_dir = Path("data/processed")

out_path = processed_dir / "transformed_diabetes_prediction_dataset.csv"
df_preprocessed.to_csv(out_path, index=False)

print("Saved to:", out_path.resolve())

df_transformed = pd.read_csv("data/processed/transformed_diabetes_prediction_dataset.csv")

print("Shape (rows, columns):", df_transformed.shape)
df_transformed.head()

del df_preprocessed

# Target variable
target = "diabetes"

# Separate X (independent variables) and y (dependent variable)

# Store all columns except the target (features used to predict)
X = df_transformed.drop(columns=[target])

# Store the target variable only (the thing to predict)
y = df_transformed[target]

print(X.shape)
print(y.shape)

# Split the transformed dataset into train and test

# X_train --> Features used to teach the model
# y_train --> The real answers (prices) for those same training rows

# X_test --> Features the model has not seen
# y_test --> The real prices for the test rows

# Split into train and test sets
# - test_size = 0.2   ---> 20% goes to test. 80% goes to train.
# - stratify = y      ---> keeps the same 0/1 ratio in both train and test
# - random_state = 42 ---> makes the split repeatable (same result every run)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

# Display the number of rows and columns in each split
print("X_train shape:", X_train.shape)
print("X_test shape :", X_test.shape)

# Check that class balance is similar
print("Train target distribution (%):")
print((y_train.value_counts(normalize=True) * 100).round(2))

print("\nTest target distribution (%):")
print((y_test.value_counts(normalize=True) * 100).round(2))

# Next step: Scaling

# Note:
# scaling just means making numbers smaller or larger so they’re on a similar range —
# without changing their meaning.
# Suppose we have these two columns (just an example)
# age           [20, 30, 40, 50]
# blood glucose [90, 150, 200, 250]

# if you don’t scale them, the model will see:
# Age → 50
# Glucose → 250

# and might think “glucose is more important because it’s bigger” —
# even though the scale difference is just units (years vs mg/dL).

# what scaling does is it re-centers and resizes the numbers so they’re comparable.

# Numeric columns to scale
numeric_cols = ["age", "bmi", "HbA1c_level", "blood_glucose_level"]


# Build a transformer that:
# - scales ONLY the numeric columns
# - leaves all other columns (the ones with values 0/1) unchanged (hypertension, heart_disease, gender, smoking dummies)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_cols)
    ],
    remainder="passthrough"  # keep other columns as they are (the ones with 0/1 values)
)

# Fit on training data ONLY, then transform train and test
X_train_scaled = preprocessor.fit_transform(X_train)
X_test_scaled  = preprocessor.transform(X_test)

# Note: Its possible, after scaling, it will affect the number of rows and columns of your trained and test sets.
# So, its a good idea to check the shape after scaling them.
# See if the shape remain the same before and after scaling.

# ----- Model 1: Logistic Regression -----
# This is a simple and popular algorithm for binary classification (0 or 1 problems)
# It tries to find a mathematical relationship between features and the target.

# Create and train the model
# 'class_weight="balanced"' helps handle the class imbalance (more 0s than 1s)
# 'max_iter=1000' increases the limit for training steps, so it can converge properly
lr = LogisticRegression(class_weight="balanced", max_iter=1000, random_state=42)

# Train (fit) the model using the training data
lr.fit(X_train_scaled, y_train)

# Notes:
# lr = LogisticRegression(class_weight="balanced", max_iter=1000, random_state=42)

# 1) class_weight='balanced'
# This tells the model:
# “If one class (for example, ‘no diabetes’) has many more samples than the other (‘diabetes’), give extra importance to the smaller class.”
# Why?
# - Because if 90% are “no diabetes” and only 10% are “diabetes”,
#   a lazy model could just predict “no diabetes” every time and still get 90% accuracy
#   So class_weight="balanced" makes it treat both sides fairly.

# 2) max_iter=1000
# the MAX number of learning steps (iterations) the solver is allowed to take.
# The model improves step-by-step; each step = 1 iteration.
# It will STOP EARLY if it finishes before 1000 (so 1000 is just an upper limit).
# If this number is too small, you may get a "ConvergenceWarning"
# (meaning it ran out of steps before finishing learning).

# 3) random_state=42
# get the same model behavior every time that line of code is run.

# Make predictions on the TEST set (data the model has never seen)

# y_pred is the model's final decision for each person → 0 = no diabetes, 1 = diabetes
y_pred = lr.predict(X_test_scaled)

# y_proba is the model's confidence for class 1 (diabetes) → a number between 0 and 1
y_proba = lr.predict_proba(X_test_scaled)[:, 1]  # gives probability of class 1 (diabetes)

# Check how good the model is (on TEST data)

print("Logistic Regression — Test metrics")

# Accuracy: out of ALL people, how many did it get right?
print("Accuracy :", round(accuracy_score(y_test, y_pred), 4))

# Precision: of the people it said "diabetes" (1), how many truly had diabetes?
print("Precision:", round(precision_score(y_test, y_pred), 4))

# Recall: of all the people who truly had diabetes (1), how many did we correctly find?
print("Recall   :", round(recall_score(y_test, y_pred), 4))

# F1-score: a single score that balances precision and recall (higher is better)
print("F1-score :", round(f1_score(y_test, y_pred), 4))

# ROC-AUC: how well the model separates 0 vs 1 across all possible thresholds (higher is better)
print("ROC-AUC  :", round(roc_auc_score(y_test, y_proba), 4))

# Confusion matrix for logistic regression
cm_lr = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(5,4))
sns.heatmap(cm_lr, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.title("Confusion Matrix — Logistic Regression")
plt.xlabel("Predicted Label")
plt.ylabel("Actual Label")

# Use the same tick labels + rotation style as MLP
plt.xticks([0.5, 1.5], ["Predicted 0 (No Diabetes)", "Predicted 1 (Diabetes)"], rotation=20)
plt.yticks([0.5, 1.5], ["Actual 0 (No Diabetes)", "Actual 1 (Diabetes)"], rotation=0)

plt.tight_layout()
plt.show()

# We have more class 0 than class 1.
# Oversampling = make extra copies of class 1 in TRAIN so both classes have similar counts.
# (Do NOT touch TEST. TEST must stay real.)
ros = RandomOverSampler(random_state=42)

# Make a new TRAIN set where 0s and 1s are balanced
X_train_bal, y_train_bal = ros.fit_resample(X_train_scaled, y_train)

# Show class counts before/after (TRAIN only)
print("Before oversampling:", y_train.value_counts().to_dict())
print("After  oversampling:", y_train_bal.value_counts().to_dict())

# ----- Model 2: Neural Network (MLP) -----

# MLPClassifier = a simple neural network.
# Think of "hidden_layer_sizes=(64, 32)" as:
# layer 1 has 64 tiny calculators, layer 2 has 32 tiny calculators.
# activation="relu"        → helps the network learn non-linear patterns.
# solver="adam"            → the method it uses to learn (a good default).
# alpha=1e-4               → a small “penalty” to avoid overfitting.
# learning_rate_init=0.001 → how big each learning step is.
# early_stopping=True      → stop training early if it stops getting better.
# n_iter_no_change=10      → “getting better” must happen within 10 checks, or stop.
# max_iter=200             → don’t train more than 200 rounds.
# random_state=42          → make results repeatable.
mlp = MLPClassifier(
    hidden_layer_sizes=(64, 32),
    activation="relu",
    solver="adam",
    alpha=1e-4,
    learning_rate_init=0.001,
    early_stopping=True,
    n_iter_no_change=10,
    max_iter=200,
    random_state=42
)

# Teach the neural network using the BALANCED TRAIN data
mlp.fit(X_train_bal, y_train_bal)

# Make predictions on TEST (data the model never saw)
# y_pred_mlp  = final answer (0 or 1)
# y_proba_mlp = confidence it is class 1 (a number between 0 and 1)
y_pred_mlp  = mlp.predict(X_test_scaled)
y_proba_mlp = mlp.predict_proba(X_test_scaled)[:, 1]

print("Neural Network (MLP) — Test metrics")

# Accuracy: out of ALL people, how many did it get right?
print("Accuracy :", round(accuracy_score(y_test, y_pred_mlp), 4))

# Precision: of the people it said "diabetes" (1), how many truly had diabetes?
print("Precision:", round(precision_score(y_test, y_pred_mlp), 4))

# Recall: of all the people who truly had diabetes (1), how many did we correctly find?
print("Recall   :", round(recall_score(y_test, y_pred_mlp), 4))

# F1-score: a single score that balances precision and recall (higher is better)
print("F1-score :", round(f1_score(y_test, y_pred_mlp), 4))

# ROC-AUC: how well the model separates 0 vs 1 across all possible thresholds (higher is better)
print("ROC-AUC  :", round(roc_auc_score(y_test, y_proba_mlp), 4))

# cm_mlp is a 2x2 table:
# [[TN, FP],
#  [FN, TP]]
# TN = predicted 0 and was 0
# FP = predicted 1 but was 0
# FN = predicted 0 but was 1
# TP = predicted 1 and was 1

# Compute confusion matrix
cm_mlp = confusion_matrix(y_test, y_pred_mlp)

# Plot heatmap
plt.figure(figsize=(5,4))
sns.heatmap(cm_mlp, annot=True, fmt="d", cmap="Oranges", cbar=False)
plt.title("Confusion Matrix — Neural Network (MLP)")
plt.xlabel("Predicted Label")
plt.ylabel("Actual Label")

# Label axes clearly for binary classification
plt.xticks([0.5, 1.5], ["Predicted 0 (No Diabetes)", "Predicted 1 (Diabetes)"], rotation=20)
plt.yticks([0.5, 1.5], ["Actual 0 (No Diabetes)", "Actual 1 (Diabetes)"], rotation=0)

plt.tight_layout()
plt.show()

# ---- Compare Logistic Regression vs MLP on key metrics (bar chart) ----

# Calculate metrics for both models
lr_metrics = {
    "Accuracy":  accuracy_score(y_test, y_pred),
    "Precision": precision_score(y_test, y_pred),
    "Recall":    recall_score(y_test, y_pred),
    "F1":        f1_score(y_test, y_pred),
    "ROC-AUC":   roc_auc_score(y_test, y_proba),
}

mlp_metrics = {
    "Accuracy":  accuracy_score(y_test, y_pred_mlp),
    "Precision": precision_score(y_test, y_pred_mlp),
    "Recall":    recall_score(y_test, y_pred_mlp),
    "F1":        f1_score(y_test, y_pred_mlp),
    "ROC-AUC":   roc_auc_score(y_test, y_proba_mlp),
}

# Prepare data for plotting
labels   = list(lr_metrics.keys())
lr_vals  = [lr_metrics[k]  for k in labels]
mlp_vals = [mlp_metrics[k] for k in labels]

x = np.arange(len(labels))
w = 0.37  # bar width

plt.figure(figsize=(8,5))
b1 = plt.bar(x - w/2, lr_vals,  width=w, label="Logistic Regression")
b2 = plt.bar(x + w/2, mlp_vals, width=w, label="Neural Net (MLP)")

# Add numbers on top of bars (4 decimal places)
for bars in (b1, b2):
    for rect in bars:
        h = rect.get_height()
        plt.text(rect.get_x() + rect.get_width()/2, h, f"{h:.4f}",
                 ha="center", va="bottom", fontsize=9)

plt.xticks(x, labels, rotation=15)
plt.ylim(0, 1.05)
plt.ylabel("Score")
plt.title("Model Performance Comparison (LR vs MLP)")
plt.legend()
plt.tight_layout()
plt.show()

# ======================================= STEP 6: MODEL TUNING =======================================

# Tune LOGISTIC REGRESSION

# Define the model again
lr = LogisticRegression(max_iter=1000, class_weight="balanced", solver="liblinear", random_state=42)

# Define the hyperparameter grid to test
# Tells the computer which settings (parameters) to test
# - 'C' controls how strict the model is:
#       small C  → simpler model (less flexible)
#       big C    → more flexible model (can overfit)
# - 'penalty' controls how the model keeps things simple:
#       'l1' → can remove less important features
#       'l2' → keeps all features but makes their effect smaller
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2']        
}

# GridSearchCV = test every combination of 'C' and 'penalty'
# - cv=5: test each combination 5 times with different train/test splits
# - scoring='f1': choose the model that has the best F1 score (balance of precision & recall)
# - n_jobs=-1: use all CPU cores (faster)
# - verbose=1: show progress while running
grid_search_lr = GridSearchCV(
    estimator=lr,
    param_grid=param_grid,
    cv=5,                 # 5-fold cross-validation
    scoring='f1',         # focus on F1 score since dataset is imbalanced
    n_jobs=-1,            # use all CPU cores
    verbose=1
)

# Train and test all the combinations on TRAIN data only
grid_search_lr.fit(X_train_scaled, y_train)

# Show the best settings and its average score
# - best_params_ : the setting (C and penalty) that worked best
# - best_score_  : how well it did on average (F1 score)print("Best parameters:", grid_search_lr.best_params_)
print("Best F1 score (cross-validation):", round(grid_search_lr.best_score_, 4))

# Rebuild Logistic Regression using the BEST settings found by GridSearch
lr_tuned = LogisticRegression(
    C=grid_search_lr.best_params_['C'],             # use the best C (picked by GridSearch)
    penalty=grid_search_lr.best_params_['penalty'], # use the best penalty ('l1' or 'l2')
    solver="liblinear",          # works with both 'l1' and 'l2' for binary problems
    class_weight="balanced",     # treat minority class fairly
    max_iter=1000,              # give it enough steps to finish learning
    random_state=42              # make results repeatable
)

# Train the model on the FULL training set
# (After GridSearch, we retrain once using all training data with the best settings.)
lr_tuned.fit(X_train_scaled, y_train)

y_pred_tuned  = lr_tuned.predict(X_test_scaled)
y_proba_tuned = lr_tuned.predict_proba(X_test_scaled)[:, 1]

# Show test scores (higher is better)
print("Logistic Regression (TUNED) — Test metrics")

# Accuracy: out of ALL people, how many did it get right?
print("Accuracy :", round(accuracy_score(y_test, y_pred_tuned), 4))

# Precision: of the people it said "diabetes" (1), how many truly had diabetes?
print("Precision:", round(precision_score(y_test, y_pred_tuned), 4))

# Recall: of all the people who truly had diabetes (1), how many did we correctly find?
print("Recall   :", round(recall_score(y_test, y_pred_tuned), 4))

# F1-score: a single score that balances precision and recall (higher is better)
print("F1-score :", round(f1_score(y_test, y_pred_tuned), 4))

# ROC-AUC: how well the model separates 0 vs 1 across all possible thresholds (higher is better)
print("ROC-AUC  :", round(roc_auc_score(y_test, y_proba_tuned), 4))

cm_tuned = confusion_matrix(y_test, y_pred_tuned)

plt.figure(figsize=(5,4))
sns.heatmap(cm_tuned, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.title("Confusion Matrix — Logistic Regression (TUNED)")
plt.xlabel("Predicted Label")
plt.ylabel("Actual Label")

# match the same axis labels/rotation we used before
plt.xticks([0.5, 1.5], ["Predicted 0 (No Diabetes)", "Predicted 1 (Diabetes)"], rotation=20)
plt.yticks([0.5, 1.5], ["Actual 0 (No Diabetes)", "Actual 1 (Diabetes)"], rotation=0)

plt.tight_layout()
plt.show()

# Notes:
# The reason the tuned Logistic Regression results look identical to the original model is simply that:
# the first Logistic Regression already used the best possible parameters.

# Both setups lead to almost the same decision boundary and same metrics, because:
# - the data is clean and scaled,
# - Logistic Regression is a simple linear model (so small changes in regularization don’t change results much),
# - and the dataset has a clear pattern, so the model already converged optimally.
# So the “tuned” version didn’t make things worse — it just confirmed your defaults were already good

# Now, tune MLP (Neural Network)

# Start with a base MLP model
# activation="relu"     → good default for non-linear patterns
# solver="adam"         → common, fast optimizer
# early_stopping=True   → stop training if it stops improving
# n_iter_no_change=10   → if no improvement for 10 checks, stop
# max_iter=200          → do not train more than 200 rounds
# random_state=42       → same results every run
mlp = MLPClassifier(
    activation="relu",
    solver="adam",
    early_stopping=True,
    n_iter_no_change=10,
    max_iter=200,
    random_state=42
)
# Tell GridSearch which settings to try
# hidden_layer_sizes : how many neurons per hidden layer
#    (64,)     → 1 hidden layer with 64 neurons
#    (64, 32)  → 2 hidden layers: 64 then 32
#    (128, 64) → 2 hidden layers: 128 then 64
#    alpha     : regularization strength (bigger = simpler model)
#    learning_rate_init  : how big each learning step is
param_grid_mlp = {
    'hidden_layer_sizes': [(64,), (64, 32), (128, 64)],
    'alpha': [1e-4, 1e-3, 1e-2],
    'learning_rate_init': [0.001, 0.01]
}
# GridSearchCV = try every combo and pick the best
# scoring='f1'  → use F1 score (good for imbalanced data)
# cv=3          → 3-fold cross-validation (fair average score)
# n_jobs=-1     → use all CPU cores (faster)
# verbose=1     → show progress
grid_search_mlp = GridSearchCV(
    estimator=mlp,
    param_grid=param_grid_mlp,
    scoring='f1',
    cv=3,
    n_jobs=-1,
    verbose=1
)
# Run the search on TRAIN data only
grid_search_mlp.fit(X_train_scaled, y_train)
# Show the best settings and its average F1 across the folds
print("Best parameters:", grid_search_mlp.best_params_)
print("Best F1 score (cross-validation):", round(grid_search_mlp.best_score_, 4))

# Rebuild the MLP with the BEST settings found
best_mlp = MLPClassifier(
    hidden_layer_sizes=(128, 64), # 2 hidden layers: first has 128 neurons, second has 64
    alpha=0.0001,                 # small regularization (helps reduce overfitting)
    learning_rate_init=0.001,     # size of each learning step
    activation="relu",            # common non-linear function (good default)
    solver="adam",                # the optimizer that updates the weights
    early_stopping=True,          # stop early if validation score stops improving
    n_iter_no_change=10,          # if no improvement for 10 checks, stop
    max_iter=200,                 # don’t train more than 200 rounds
    random_state=42               # same results every run
)

# Train the model using the TRAIN data
best_mlp.fit(X_train_scaled, y_train)

# Make predictions on the TEST data (unseen during training)
# y_pred_best  = final class (0 or 1)
# y_proba_best = probability of class 1 (diabetes) for each persony_pred_best = best_mlp.predict(X_test_scaled)
y_pred_best  = best_mlp.predict(X_test_scaled)
y_proba_best = best_mlp.predict_proba(X_test_scaled)[:, 1]

# Show how good the model is (higher is better)
print("Neural Network (TUNED) — Test metrics")

# Accuracy: out of ALL people, how many did it get right?
print("Accuracy :", round(accuracy_score(y_test, y_pred_best), 4))

# Precision: of the people it said "diabetes" (1), how many truly had diabetes?
print("Precision:", round(precision_score(y_test, y_pred_best), 4))

# Recall: of all the people who truly had diabetes (1), how many did we correctly find?
print("Recall   :", round(recall_score(y_test, y_pred_best), 4))

# F1-score: a single score that balances precision and recall (higher is better)
print("F1-score :", round(f1_score(y_test, y_pred_best), 4))

# ROC-AUC: how well the model separates 0 vs 1 across all possible thresholds (higher is better)
print("ROC-AUC  :", round(roc_auc_score(y_test, y_proba_best), 4))

cm_best = confusion_matrix(y_test, y_pred_best)

plt.figure(figsize=(5,4))
sns.heatmap(cm_best, annot=True, fmt="d", cmap="Oranges", cbar=False)
plt.title("Confusion Matrix — Neural Network (TUNED)")
plt.xlabel("Predicted Label")
plt.ylabel("Actual Label")

# match the same axis labels/rotation as before
plt.xticks([0.5, 1.5], ["Predicted 0 (No Diabetes)", "Predicted 1 (Diabetes)"], rotation=20)
plt.yticks([0.5, 1.5], ["Actual 0 (No Diabetes)", "Actual 1 (Diabetes)"], rotation=0)

plt.tight_layout()
plt.show()

# Metric values from your results
logreg_scores = {
    "Accuracy": 0.8851,
    "Precision": 0.4264,
    "Recall": 0.8762,
    "F1-score": 0.5736,
    "ROC-AUC": 0.9596
}

mlp_scores = {
    "Accuracy": 0.9703,
    "Precision": 0.9569,
    "Recall": 0.6946,
    "F1-score": 0.8049,
    "ROC-AUC": 0.9746
}

# Extract metric names and values
metrics = list(logreg_scores.keys())
logreg_vals = list(logreg_scores.values())
mlp_vals = list(mlp_scores.values())

# Bar positions
x = np.arange(len(metrics))
width = 0.35

# Plot
plt.figure(figsize=(8,5))
bars1 = plt.bar(x - width/2, logreg_vals, width, label='Logistic Regression (Tuned)', color='#4A90E2')
bars2 = plt.bar(x + width/2, mlp_vals, width, label='Neural Network (Tuned)', color='#F5A623')

# Labels and formatting
plt.xlabel("Evaluation Metrics")
plt.ylabel("Score")
plt.title("Model Performance Comparison — Logistic Regression vs Neural Network")
plt.xticks(x, metrics)
plt.ylim(0, 1.1)
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.6)

# Display bars’ numeric values
for bars in [bars1, bars2]:
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval + 0.02, f"{yval:.2f}",
                 ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

